# Algorithm Testing
A notebook used to compare algorithm performance.


In [2]:
### Imports
from nitools import utils 
from nitools.classifiers import PAE_RVFL, ELM, AE_ELM, PAE_ELM
from nitools.models import LeNet5, ResNet18
from nitools.operations import resetseed

import numpy as np
import random
import torch
import time as t

## 1. MNIST

Dataset of greyscale 28x28 images of handwritten digits. 

*Train: 60,000*, *Test: 10,000*, *10 Classes*, *784 Inputs*


In [3]:
# Import dataset
mnist = utils.load_mnist()

mnist_in = 784
mnist_class = 10

### pAE-RVFL

In [7]:
device = torch.device("cpu")

SEED = 22
resetseed(SEED)

pae_rvfl = PAE_RVFL(
    784, 
    200, 
    mnist_class, 
    subnets=1,
    device=device, 
    r=(1,1), sc=0.5, sb=0.5, c=10, ae_iters=1)

lenet = LeNet5(pae_rvfl, weight_train=False)

X = torch.from_numpy(mnist['train_X']).float()[:60000]
y = torch.from_numpy(mnist['train_y']).float()[:60000]

train_start = t.time()
result = lenet.train(X, y)
train_end = t.time()
tX = torch.from_numpy(mnist['test_X']).float()
pred = lenet.predict(tX)[:10000]
pred_arg = np.zeros(10000)

for i in range(len(pred)):
    pred_arg[i] = np.argmax(pred[i])

print(f"train: {train_end-train_start}")
utils.evaluation_summary('lenet PAE-RVFL - MNIST', pred_arg, mnist['test_y'])

train: 2.2266740798950195
Evaluation for: lenet PAE-RVFL - MNIST
Classifier 'lenet PAE-RVFL - MNIST' has Acc=0.930 P=0.929 R=0.930 F1=0.930
              precision    recall  f1-score   support

         0.0      0.978     0.947     0.962      1012
         1.0      0.989     0.935     0.961      1201
         2.0      0.905     0.943     0.924       990
         3.0      0.901     0.926     0.913       983
         4.0      0.936     0.935     0.935       983
         5.0      0.883     0.940     0.911       838
         6.0      0.959     0.937     0.948       981
         7.0      0.914     0.935     0.925      1005
         8.0      0.908     0.902     0.905       980
         9.0      0.920     0.904     0.912      1027

    accuracy                          0.930     10000
   macro avg      0.929     0.930     0.930     10000
weighted avg      0.931     0.930     0.931     10000


Confusion matrix:
 [[ 958    1    2    0    3    0    8    1    7    0]
 [   0 1123    5    0    0  

### ELM

In [9]:
SEED = 22
resetseed(SEED)

# Get ELM results
mnist_elm = AE_ELM.AE_ELM(  # TODO: batchnorm input weights to ae?
    input_size=mnist_in,
    h_size=100,
    output_size=mnist_class,
    ae_iters=3)

lenet = ResNet18.ResNet18(mnist_elm, weight_train=True)

X = torch.from_numpy(mnist['train_X']).float()[:5000]
y = torch.from_numpy(mnist['train_y']).float()[:5000]

train_start = t.time()
result = lenet.train(X, y)
train_end = t.time()
tX = torch.from_numpy(mnist['test_X']).float()
pred = lenet.predict(tX)[:10000]
pred_arg = np.zeros(10000)

for i in range(len(pred)):
    pred_arg[i] = np.argmax(pred[i])


print(f"train: {train_end-train_start}")
utils.evaluation_summary('lenet AE-ELM - MNIST', pred_arg, mnist['test_y'])

RuntimeError: The size of tensor a (10) must match the size of tensor b (6) at non-singleton dimension 3

In [20]:
SEED = 22
resetseed(SEED)

# Get ELM results
mnist_elm = PAE_ELM(  # TODO: batchnorm input weights to ae?
    in_size=mnist_in,
    h_size=100,
    subnets=5,
    out_size=mnist_class)

lenet = mnist_elm#LeNet5.LeNet5(mnist_elm, weight_train=True)

X = torch.from_numpy(mnist['train_X']).float()[:60000]
y = torch.from_numpy(mnist['train_y']).float()[:60000]

train_start = t.time()
result = lenet.train(X, y)
train_end = t.time()
tX = torch.from_numpy(mnist['test_X']).float()
pred = lenet.predict(tX)[:10000]
pred_arg = np.zeros(10000)

for i in range(len(pred)):
    pred_arg[i] = np.argmax(pred[i])


print(f"train: {train_end-train_start}")
utils.evaluation_summary('lenet  - MNIST', pred_arg, mnist['test_y'])

train: 2.3136589527130127
Evaluation for: lenet  - MNIST
Classifier 'lenet  - MNIST' has Acc=0.916 P=0.915 R=0.916 F1=0.915
              precision    recall  f1-score   support

         0.0      0.981     0.949     0.964      1013
         1.0      0.984     0.920     0.951      1214
         2.0      0.896     0.953     0.924       971
         3.0      0.903     0.914     0.908       998
         4.0      0.915     0.888     0.902      1012
         5.0      0.867     0.919     0.892       841
         6.0      0.954     0.924     0.939       989
         7.0      0.905     0.906     0.905      1027
         8.0      0.873     0.908     0.890       936
         9.0      0.875     0.884     0.879       999

    accuracy                          0.916     10000
   macro avg      0.915     0.916     0.915     10000
weighted avg      0.918     0.916     0.917     10000


Confusion matrix:
 [[ 961    0    0    1    0    6    9    1    2    0]
 [   0 1117    2    2    1    2    5    0   

## 2. CIFAR-10

Dataset of colour (3-dimension) 32x32 images of objects of 10 classes:

0.	airplane
1.	automobile
2.	bird
3.	cat
4.	deer
5.	dog
6.	frog
7.	horse
8.	ship
9.	truck

*Train: 60,000*, *Test: 10,000*, *10 Classes*, *784 Inputs*

In [ ]:
# Import dataset
cifar10 = utils.load_cifar10()

cifar_in = 32*32*3
cifar_class = 10

### SPAE-RVFL

In [ ]:
device = torch.device("cpu")

pae_rvfl = SPAE_RVFL.SPAE_RVFL(
    cifar_in, 
    cifar_class, 
    [100], 
    subnets=[5], 
    device=device, 
    r=[(1,1), (1,1)], sc=0.5, sb=0.5)

lenet = LeNet5.LeNet5_3D(pae_rvfl)

X = torch.from_numpy(cifar10['train_X']).float()[:5000]
y = torch.from_numpy(cifar10['train_y']).float()[:5000]

result = lenet.train(X, y)
tX = torch.from_numpy(cifar10['test_X']).float()
pred = lenet.predict(tX)[:10000]
pred_arg = np.zeros(10000)

for i in range(len(pred)):
    pred_arg[i] = np.argmax(pred[i])

utils.evaluation_summary('lenet PAE-RVFL - MNIST', pred_arg, cifar10['test_y'])

In [ ]:
# Show results

In [29]:
device = torch.device("cpu")

SEED = 22
resetseed(SEED)

pae_elm = PAE_ELM(
    784, 
    300, 
    mnist_class, 
    subnets=5,
    device=device, 
    c=10, ae_iters=3)

lenet = LeNet5(pae_elm, weight_train=False)

X = torch.from_numpy(mnist['train_X']).float()[:60000]
y = torch.from_numpy(mnist['train_y']).float()[:60000]

train_start = t.time()
result = lenet.train(X, y)
train_end = t.time()
tX = torch.from_numpy(mnist['test_X']).float()
pred = lenet.predict(tX)[:10000]
pred_arg = np.zeros(10000)

for i in range(len(pred)):
    pred_arg[i] = np.argmax(pred[i])

print(f"train: {train_end-train_start}")
utils.evaluation_summary('lenet PAE-ELM - MNIST', pred_arg, mnist['test_y'])

train: 7.716118335723877
Evaluation for: lenet PAE-ELM - MNIST
Classifier 'lenet PAE-ELM - MNIST' has Acc=0.970 P=0.969 R=0.969 F1=0.969
              precision    recall  f1-score   support

         0.0      0.989     0.976     0.982       993
         1.0      0.989     0.986     0.988      1139
         2.0      0.968     0.969     0.968      1031
         3.0      0.966     0.966     0.966      1010
         4.0      0.973     0.978     0.975       976
         5.0      0.967     0.963     0.965       896
         6.0      0.971     0.979     0.975       950
         7.0      0.953     0.969     0.961      1011
         8.0      0.970     0.947     0.958       998
         9.0      0.946     0.959     0.953       996

    accuracy                          0.970     10000
   macro avg      0.969     0.969     0.969     10000
weighted avg      0.970     0.970     0.970     10000


Confusion matrix:
 [[ 969    0    0    1    1    1    3    1    3    1]
 [   0 1123    3    1    0    1